In [9]:
import os
import glob
import torch
import nibabel as nib
import numpy as np
from monai.transforms import (
    Compose, LoadImage, EnsureChannelFirst, Spacing, Orientation,
    ScaleIntensityRange, CropForeground, EnsureType, DivisiblePad, Resize
)
from monai.networks.nets import AttentionUnet
from monai.inferers import SlidingWindowInferer

In [10]:
def run_inference():
    # Cấu hình đường dẫn
    model_path = "/mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/saved_models/best_model.pth"
    input_folder = "/mrhung_nguyen_minh_quang_108/workspace/train/nnUNet_raw/Dataset015_lungTumor/imagesTs"
    output_folder = "/mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference"

    os.makedirs(output_folder, exist_ok=True)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Sử dụng thiết bị: {device}")

    # Khởi tạo mô hình Attention UNet
    model = AttentionUnet(
        spatial_dims=3,
        in_channels=1,
        out_channels=1,
        channels=(16, 32, 64, 128, 256),
        strides=(2, 2, 2, 2),
    ).to(device)

    if not os.path.exists(model_path):
        raise FileNotFoundError(f"Không tìm thấy file model: {model_path}")

    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()
    print(f"Đã nạp model từ: {model_path}")

    # Transforms cho inference
    inference_transforms = Compose([
        LoadImage(image_only=True),
        EnsureChannelFirst(),
        Spacing(pixdim=(2.0, 2.0, 2.0), mode="bilinear"),
        Orientation(axcodes="RAS"),
        ScaleIntensityRange(a_min=-1000, a_max=400, b_min=0.0, b_max=1.0, clip=True),
        CropForeground(source_key="image", allow_smaller=True),
        DivisiblePad(k=16),
        EnsureType()
    ])

    inferer = SlidingWindowInferer(
        roi_size=[96, 96, 96],
        sw_batch_size=4,
        overlap=0.5,
    )

    input_files = sorted(glob.glob(os.path.join(input_folder, "*.nii.gz")))
    if len(input_files) == 0:
        print(f"Không tìm thấy file .nii.gz nào trong thư mục: {input_folder}")
        return

    print(f"Tìm thấy {len(input_files)} file cần xử lý...")

    for idx, input_file in enumerate(input_files):
        print(f"Đang xử lý file {idx+1}/{len(input_files)}: {os.path.basename(input_file)}")

        try:
            # Load ảnh gốc để lấy shape, affine, header
            original_img = nib.load(input_file)
            original_shape = original_img.shape
            original_affine = original_img.affine
            original_header = original_img.header

            # Tiền xử lý
            input_image = inference_transforms(input_file)
            input_tensor = input_image.unsqueeze(0).to(device)

            with torch.no_grad():
                with torch.cuda.amp.autocast():
                    output = inferer(input_tensor, model)
                    output = torch.sigmoid(output)
                    binary_output = (output > 0.5).float()

            binary_output = binary_output.squeeze().cpu().numpy().astype(np.uint8)

            # === Khôi phục về shape gốc như UNet ===
            resized_output = Resize(spatial_size=original_shape, mode="nearest")(binary_output[None])[0].numpy().astype(np.uint8)

            # Lưu segmentation mask
            output_filename = os.path.basename(input_file).replace(".nii.gz", "_atten_seg.nii.gz")
            output_file = os.path.join(output_folder, output_filename)

            output_img = nib.Nifti1Image(resized_output, affine=original_affine, header=original_header)
            nib.save(output_img, output_file)

            print(f"Đã lưu segmentation tại: {output_file}")

        except Exception as e:
            print(f"Lỗi khi xử lý file {input_file}: {str(e)}")

    print(f"Đã hoàn thành inference cho {len(input_files)} file. Kết quả được lưu tại: {output_folder}")

In [11]:
if __name__ == "__main__":
    run_inference()

Sử dụng thiết bị: cuda


/tmp/ipykernel_9505/1212735125.py:23: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location=device))


Đã nạp model từ: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/saved_models/best_model.pth
Tìm thấy 29 file cần xử lý...
Đang xử lý file 1/29: lung_001_0000.nii.gz


/tmp/ipykernel_9505/1212735125.py:67: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():


Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_001_0000_atten_seg.nii.gz
Đang xử lý file 2/29: lung_007_0000.nii.gz
Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_007_0000_atten_seg.nii.gz
Đang xử lý file 3/29: lung_008_0000.nii.gz
Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_008_0000_atten_seg.nii.gz
Đang xử lý file 4/29: lung_010_0000.nii.gz
Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_010_0000_atten_seg.nii.gz
Đang xử lý file 5/29: lung_024_0000.nii.gz
Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_024_0000_atten_seg.nii.gz
Đang xử lý file 6/29: lung_025_0000.nii.gz
Đã lưu segmentation tại: /mrhung_nguyen_minh_quang_108/workspace/AttenUNet_2/inference/lung_025_0000_atten_seg.nii.gz
Đang xử lý file 7/29: lung_028_0000.nii.gz
Đã lưu segmentation tại: /mrhung_n